In [1]:
from pyspark import SparkConf, SparkContext
conf = SparkConf().setMaster('local').setAppName('my-test-conf')
sc = SparkContext(conf=conf)

24/07/08 17:27:19 WARN Utils: Your hostname, MZC01-HYUCKSANGCHO.local resolves to a loopback address: 127.0.0.1; using 10.90.8.177 instead (on interface en0)
24/07/08 17:27:19 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/07/08 17:27:20 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


SparkConf : 사용자가 재정의해서 쓸 수 있는 설정 옵션  
SparkContext : Spark 클러스터와 연결시켜주는 객체(자바로 동작하는 py4J의 SparkContext와 연결)

### Key-Value RDD

In [36]:
lines = sc.textFile('/Users/mzc01-hyucksangcho/Desktop/spark/data-engineering/01-spark/data/restaurant_reviews.csv')
lines.collect()

['id,item,category,reviews',
 '0,짜장면 ,중식 ,125',
 '1,짬뽕 ,중식 ,235',
 '2,김밥 ,분식 ,32',
 '3,떡볶이 ,분식 ,534',
 '4,라멘 ,일식 ,223',
 '5,돈가스 ,일식 ,52',
 '6,우동 ,일식 ,12 ']

In [37]:
header = lines.first()
lines = lines.filter(lambda row : row!= header)
lines.collect()

['0,짜장면 ,중식 ,125',
 '1,짬뽕 ,중식 ,235',
 '2,김밥 ,분식 ,32',
 '3,떡볶이 ,분식 ,534',
 '4,라멘 ,일식 ,223',
 '5,돈가스 ,일식 ,52',
 '6,우동 ,일식 ,12 ']

In [39]:
def parse(row):
    row = row.split(',')
    category = row[2][:-1]
    reviews = (int(row[3]), 1)
    return (category, reviews)

parse_lines = lines.map(parse)
parse_lines.collect()

[('중식', (125, 1)),
 ('중식', (235, 1)),
 ('분식', (32, 1)),
 ('분식', (534, 1)),
 ('일식', (223, 1)),
 ('일식', (52, 1)),
 ('일식', (12, 1))]

In [41]:
category_collect = parse_lines.reduceByKey(lambda a, b : (a[0]+b[0], a[1]+b[1]))
category_collect.collect()

[('중식', (360, 2)), ('분식', (566, 2)), ('일식', (287, 3))]

In [43]:
category_collect.mapValues(lambda x : x[0]/x[1]).collect()

[('중식', 180.0), ('분식', 283.0), ('일식', 95.66666666666667)]

### RDD Transformaton과 Action

In [2]:
foods = sc.parallelize(["짜장면", "마라탕", "짬뽕", "떡볶이", "쌀국수", "짬뽕", "짜장면", "짜장면", "짜장면",  "라면", "우동", "라면"])
foods.collect()

['짜장면', '마라탕', '짬뽕', '떡볶이', '쌀국수', '짬뽕', '짜장면', '짜장면', '짜장면', '라면', '우동', '라면']

In [53]:
foods.countByValue()

defaultdict(int,
            {'짜장면': 4,
             '마라탕': 1,
             '짬뽕': 2,
             '떡볶이': 1,
             '쌀국수': 1,
             '라면': 2,
             '우동': 1})

In [59]:
foods.take(2)

['짜장면', '마라탕']

In [60]:
foods.first()

'짜장면'

In [61]:
foods.count()

12

In [65]:
foods.distinct().collect()

['짜장면', '마라탕', '짬뽕', '떡볶이', '쌀국수', '라면', '우동']

In [7]:
foods.foreach(lambda x : print(x))

짜장면
마라탕
짬뽕
떡볶이
쌀국수
짬뽕
짜장면
짜장면
짜장면
라면
우동
라면


In [9]:
movies = [
    "그린 북",
    "매트릭스",
    "토이 스토리",
    "캐스트 어웨이",
    "포드 V 페라리",
    "보헤미안 랩소디",
    "빽 투 더 퓨처",
    "반지의 제왕",
    "죽은 시인의 사회"
]

moviesRDD = sc.parallelize(movies)
moviesRDD.collect()

['그린 북',
 '매트릭스',
 '토이 스토리',
 '캐스트 어웨이',
 '포드 V 페라리',
 '보헤미안 랩소디',
 '빽 투 더 퓨처',
 '반지의 제왕',
 '죽은 시인의 사회']

In [10]:
moviesRDD.flatMap(lambda x : x.split()).collect()

['그린',
 '북',
 '매트릭스',
 '토이',
 '스토리',
 '캐스트',
 '어웨이',
 '포드',
 'V',
 '페라리',
 '보헤미안',
 '랩소디',
 '빽',
 '투',
 '더',
 '퓨처',
 '반지의',
 '제왕',
 '죽은',
 '시인의',
 '사회']

In [13]:
foods = sc.parallelize(["짜장면", "마라탕", "짬뽕", "떡볶이", "쌀국수", "짬뽕", "짜장면", "짜장면", "짜장면",  "라면", "우동", "라면", "치킨", "돈까스", "회", "햄버거", "피자"])

In [14]:
foods.collect()

['짜장면',
 '마라탕',
 '짬뽕',
 '떡볶이',
 '쌀국수',
 '짬뽕',
 '짜장면',
 '짜장면',
 '짜장면',
 '라면',
 '우동',
 '라면',
 '치킨',
 '돈까스',
 '회',
 '햄버거',
 '피자']

In [16]:
foods.groupBy(lambda x : x[0]).collect()


[('짜', <pyspark.resultiterable.ResultIterable at 0x7ff030e4f310>),
 ('마', <pyspark.resultiterable.ResultIterable at 0x7ff030e4f190>),
 ('짬', <pyspark.resultiterable.ResultIterable at 0x7ff030e4f040>),
 ('떡', <pyspark.resultiterable.ResultIterable at 0x7ff030e4f430>),
 ('쌀', <pyspark.resultiterable.ResultIterable at 0x7ff030e4f0a0>),
 ('라', <pyspark.resultiterable.ResultIterable at 0x7ff030e4f7c0>),
 ('우', <pyspark.resultiterable.ResultIterable at 0x7ff030e4f940>),
 ('치', <pyspark.resultiterable.ResultIterable at 0x7ff030e4f4c0>),
 ('돈', <pyspark.resultiterable.ResultIterable at 0x7ff030e4fbe0>),
 ('회', <pyspark.resultiterable.ResultIterable at 0x7ff030e4f5e0>),
 ('햄', <pyspark.resultiterable.ResultIterable at 0x7ff030e4fe20>),
 ('피', <pyspark.resultiterable.ResultIterable at 0x7ff030e4f7f0>)]